In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder , OneHotEncoder
import pickle

In [35]:
## Load the Churn data
data = pd.read_csv(r"C:\Users\nahid\Desktop\DeepLearning Project\Data\Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [36]:
## Prepocess the data 
## Drop irrelevent columns 
data = data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Encode categorical variables

In [37]:
## Encode categorical variables 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


## Onehot encode 'Geography' column  Operation 

In [38]:
## onehot encode 'Geography' column 
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])

In [39]:
# to check the new columns name after onehot encoder  
geo_encoder_columns = onehot_encoder_geo.get_feature_names_out(['Geography'])

In [40]:
# First we need to condert the geo_encoder to array then we will abale to create a dataframe. 
geo_encoder_array = geo_encoder.toarray()

In [41]:
geo_encoder_df = pd.DataFrame(geo_encoder_array, columns= geo_encoder_columns)
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [42]:
## Combine one hot encoder columns with the original data
data= pd.concat([data.drop('Geography', axis=1), geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [43]:
## Save the encoders and sscaler 
with open('label_encoder_gender.pkl','wb') as file :
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file :
    pickle.dump(onehot_encoder_geo,file)

In [44]:
## Divide the dataset into independent and dependent featuers 
X= data.drop(['Exited'],axis=1)
y=data['Exited']

In [45]:
## Split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [46]:
## Scale these featuers
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [47]:
## Save the scaler file in the pkl formet 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN Implementetion 

In [48]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [49]:
X_train.shape[1]

12

In [50]:
## Build our ANN  model 
model = Sequential(
    [
        Dense(16,activation='relu',input_shape =(X_train.shape[1],)), ## HL1 connect with input layer
        Dense(16,activation='relu'), #HL2
        Dense(1, activation='sigmoid') # Output layer 
    ] 
)

c:\Users\nahid\Desktop\DeepLearning Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 16)             │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 497 (1.94 KB)

 Trainable params: 497 (1.94 KB)

 Non-trainable params: 0 (0.00 B)

In [52]:
## Example you can change the learning rate and create customiz the noptimizer 
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()

In [53]:
## Compile the model 
model.compile(optimizer="Adam",loss='binary_crossentropy',metrics=['accuracy'])

In [54]:
## Set up TensorBord
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [55]:
## Set up Early Stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss',patience= 10, restore_best_weights= True)

In [56]:
## Training the model 
history = model.fit(
    X_train,y_train, validation_data = (X_test,y_test), epochs = 50,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5777 - loss: 0.6573 - val_accuracy: 0.8030 - val_loss: 0.4487
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.7888 - loss: 0.4648 - val_accuracy: 0.8135 - val_loss: 0.4175
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.8197 - loss: 0.4080 - val_accuracy: 0.8260 - val_loss: 0.4065
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step - accuracy: 0.8244 - loss: 0.4132 - val_accuracy: 0.8325 - val_loss: 0.3926
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8265 - loss: 0.4043 - val_accuracy: 0.8445 - val_loss: 0.3755
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8384 - loss: 0.3799 - val_accuracy: 0.8590 - val_loss: 0.3591
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8500 - loss: 0.3692 - val_accuracy: 0.8600 - val_loss: 0.3480
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8544 - loss: 0.3529 - val_accur

In [57]:
model.save('model.h5')

In [58]:
## load Tensorboard Extension 
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [64]:
%tensorboard --logdir log/fit20240917-224959/

Reusing TensorBoard on port 6018 (pid 30660), started 0:00:04 ago. (Use '!kill 30660' to kill it.)